In [1]:
import matplotlib.pyplot as plt
from albumentations import Flip, ShiftScaleRotate, RandomRotate90, RandomCrop
from src.datasets import HEDataset,ConsepDataset,ConsepDataset_Fast
import os 
import torch
os.environ['CUDA_VISIBLE_DEVICES'] = "5"
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
torch.set_num_threads(3)


/home/wzy/.conda/envs/general/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.6 of module '_watershed_cy' does not match runtime version 3.7
  return f(*args, **kwds)


In [2]:
import albumentations as A
import random
import numpy as np


In [3]:
data_transform = A.Compose([A.Flip(p=0.3),
                            A.Affine(p=0.3),
                            A.Rotate(p=0.3),
                            A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=15, val_shift_limit=10, p=0.3),
                            A.GaussianBlur(3, p=0.3),
                            A.GaussNoise(30, p=0.3)],
                            
                            keypoint_params=A.KeypointParams(format='yx'),
                            additional_targets={'mask0': 'mask',
                                                'mask1': 'mask',
                                                'mask2': 'mask'})

/home/wzy/.conda/envs/general/lib/python3.7/site-packages/albumentations/augmentations/blur/transforms.py:185: UserWarning: blur_limit and sigma_limit minimum value can not be both equal to 0. blur_limit minimum value changed to 3.
  "blur_limit and sigma_limit minimum value can not be both equal to 0. "


#### Training Process

In [4]:
import glob,os
from trainval import trainval
import matplotlib.pyplot as plt
from haven import haven_utils as hu

In [5]:
exp_dict = hu.load_json('Experiment_Exp/exp_config_mynet_1.json')
exp_dict["batch_size"] = 96
exp_dict["max_epoch"] = 10

datadir = './CoNSeP/' 
random_seed = random.randint(0, 20201009)
train_set = ConsepDataset_Fast(data_dir=datadir,
                            n_classes=exp_dict["n_classes"],


                            
                            transform=data_transform,
                            option="Train",
                            random_seed=random_seed,
                            obj_option=None,
                            patch_size=256,
                            bkg_option=None)

In [6]:
# folddir_10 = glob.glob(os.path.join(datadir, '*_fold'))

# for folddir in folddir_10:
#     savedir_base = os.path.join(folddir,'Result')
#     os.makedirs(savedir_base,exist_ok=True)

savedir_base = "./Result"
os.makedirs(savedir_base,exist_ok=True)
trainval(exp_dict, savedir_base, datadir, reset=False, num_workers=8)

Experiment saved in ./Result/865a15c2d5809452aa3daad9f1b002e9


/home/wzy/.conda/envs/general/lib/python3.7/site-packages/albumentations/augmentations/blur/transforms.py:185: UserWarning: blur_limit and sigma_limit minimum value can not be both equal to 0. blur_limit minimum value changed to 3.
  "blur_limit and sigma_limit minimum value can not be both equal to 0. "


Starting experiment at epoch 0


Training Loss: 0.5381, det_loss:  0.0188, seg_loss:  0.4366, cam_pos:  0.0571, cam_neg:  0.0256:   5%|▍         | 33/675 [00:27<07:31,  1.42it/s]

KeyboardInterrupt: 

#### Testing and Visualize

In [ ]:
from src import models
from src.datasets import HEDataset,ConsepDataset
# from trainval import loadsplittedfolds
import albumentations as A
import os,glob
import torch

datadir = './CoNSeP/' 
# folddir_10 = glob.glob(os.path.join(datadir, '*_fold'))

# folddir = folddir_10[0]

exp_dict = hu.load_json('Experiment_Exp/exp_config_mynet_1.json')

test_transform = A.Compose([A.Resize(256,256)], 
                          keypoint_params = A.KeypointParams(format='yx'), 
                          additional_targets = {'mask0':'mask', 
                                                'mask1':'mask'})

# tr_idx, val_idx, ts_idx = loadsplittedfolds(folddir)
test_set = test_set = ConsepDataset(data_dir = datadir, 
                                transform = test_transform,
                                option = "Test")

model = models.get_model(exp_dict['model'],exp_dict = exp_dict,train_set = test_set)
model.load_state_dict(torch.load('Result/v33/model_best.pth'))

In [ ]:
index = 2
test_im = test_set[index]['images'][None]
model.eval()
test_logit,regression,classification = model.model_base(test_im.to(model.device))


#### 分割结果的可视化

In [ ]:
test_prob = test_logit.sigmoid()
test_seg = torch.argmax(test_prob, dim = 1)
fig, axes = plt.subplots(1,3, figsize = (30,10))
axes[0].imshow(test_im[0].detach().cpu().numpy().transpose(1,2,0))
axes[1].imshow(test_set[index]['gt'],vmax = 7, vmin = 0)
axes[2].imshow(test_seg[0].detach().cpu().numpy(),vmax = 7, vmin = 0)